# Memory in Agent

This notebook goes over adding memory to an Agent. Before going through this notebook, please walkthrough the following notebooks, as this will build on top of both of them:

- [Memory in LLMChain](/docs/modules/memory/adding_memory)
- [Custom Agents](/docs/modules/agents/how_to/custom_agent)

In order to add a memory to an agent we are going to perform the following steps:

1. We are going to create an `LLMChain` with memory.
2. We are going to use that `LLMChain` to create a custom Agent.

For the purposes of this exercise, we are going to create a simple custom Agent that has access to a search tool and utilizes the `ConversationBufferMemory` class.

In [ ]:
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import GoogleSearchAPIWrapper
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"] = "GOOGLE_CSE_ID"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

Notice the usage of the `chat_history` variable in the `PromptTemplate`, which matches up with the dynamic key name in the `ConversationBufferMemory`.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain_openai import OpenAI

llm=OpenAI(temperature=0)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

In [ ]:
agent_executor.invoke({"input":"How many people live in canada?"})

To test the memory of this agent, we can ask a followup question that relies on information in the previous exchange to be answered correctly.

In [ ]:
agent_executor.invoke({"input":"what is their national anthem called?"})

We can see that the agent remembered that the previous question was about Canada, and properly asked Google Search what the name of Canada's national anthem was.

For fun, let's compare this to an agent that does NOT have memory.

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor_without_memory = AgentExecutor(agent=agent, tools=tools)

In [ ]:
agent_executor_without_memory.invoke({"input":"How many people live in canada?"})

In [ ]:
agent_executor_without_memory.invoke({"input":"what is their national anthem called?"})